# Module 4 - Lab 1: Building a Regression Model with Spark

This lab will build upon the previous lab. 
While classification models deal with outcomes that represent discrete classes, regression models are concerned with target variables that can take any real value. Our goal is to find a model that maps input features to predicted target variables. 
Like classification, regression is also a form of supervised learning.

We can use regression models to predict a mulititude of different variables of interest. 
Here are some examples of how this technique is used in society:
* Predicting stock returns and other economic variables
* Predicting loss amounts for loan defaults (this can be combined with a classification model that predicts the probability of default, while the regression model predicts the amount in the case of a default)
* Predicting customer lifetime value (CLTV) in a retail, mobile, or other business, based on user behavior and spending patterns

Below, we will fire up our Spark cluster and prepare it for data visualization like we have done in previous labs. See Module 3 - Lab 1: Feature-Extraction-MoreContext for a detailed description of what is happening below. 

In [1]:
from pyspark.mllib.regression import LabeledPoint
import numpy as np

from pyspark.mllib.regression import LinearRegressionWithSGD
from pyspark.mllib.tree import DecisionTree

# Make Random String
import os, random, string
length = 32
chars = string.ascii_letters + string.digits + '_-'
random.seed = (os.urandom(1024))

rndfolder = ''.join(random.choice(chars) for i in range(length))
dirpath = '/home/hadoop/work_dir/' + rndfolder + '/'

# Set Path and permissions ("0770", which means everyone can read and write the file)
os.mkdir(dirpath, 0770)
os.chdir(dirpath)

def process_figure(fig, name):
    fig.savefig(name)
    print 'http://ec2-54-153-99-19.us-west-1.compute.amazonaws.com:8810/' + rndfolder + '/' + name

import matplotlib
matplotlib.use('agg') # non-graphical mode (pngs(?))
import matplotlib.pyplot as plt

import numpy as np

print("Ready")

Creating SparkContext as 'sc'


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
270,None,pyspark,idle,,,✔


Creating HiveContext as 'sqlContext'
SparkContext and HiveContext created. Executing user code ...
Ready

## Types of regression models
Let's talk about the different types of regression models before we dive into coding up an example.
Spark's MLlib library offers two broad classes of regression models: linear models and decision tree regression models.
Linear regression models use a different loss function, related link function, and decision function than its classification counterparts.
(We will go over these functions in minute.)
MLlib provides a standard least squares regression model, which is a type of a linear model.

### Least square regression
There is a variety of loss functions that can be applied to generalized linear models. 
(A loss function is a function that maps an event (or values of one or more variables) onto a real number intuitively representing some "cost" associated with the event.)

This is the loss function used for least squares (a.k.a. the squared loss):
    1/2[(w^T) * x - y]^2
* y: the target variable (it has a real value)
* w: weight vector
* x: feature vector
* T: instance? 

So what is this useful for? 
What does all of this actually mean?
Least Squares Regression is the method for summarizing a pattern in a dataset that suggests a linear relationship.
This allows us to predict an outcome, y, for a given input, x. 
(i.e. We can describe (or predict) how a response variable, y, changes as an explanatory variable, x, changes.)
This model is useful for SPECIFIC situations.
The standard least squares regression in MLlib does not use regularization (which is a process of introducing additional information in order to solve an ill-posed problem or to prevent overfitting).
We can see that the loss applied to incorrectly predicted points will be magnified since the loss is squared.
This means that least squares regression is susceptible to outliers in the dataset and also to over-fitting.
So, generally, we should apply some level of regularization to account for this.

Here are some things that may happen using this technique against the data (so if you aren't sure what the data looks like and you use this method, the following situations could occur):
* A curved pattern might appear showing that the relationship is not linear
* Increasing or decreasing spread about the line as x increases indicates that prediction of y will be LESS accurate for larger x's.
* Individual points with large residuals are outliers  in the vertical direction
* Individual points that are extreme in the x direction are also important....as influential observations
This link is helpful for explaining this concept even further: http://www.henry.k12.ga.us/ugh/apstat/chapternotes/sec3.3.html
(I recommend reading it. It's not too long, and I think it is helpful to explain the math behind the method.)

The link function is the identity link.
The decision function is also the identity function, since generally, no thresholding is applied in regression. 
Therefore, the model's prediction is simply: y = (w^T) * x

Linear regression with L2 regularization is commonly referred to as ridge regression, while applying L1 regularization is called the lasso.
To understand this concept better, read the "Regularization" section of this website: http://www.chioka.in/differences-between-l1-and-l2-as-loss-function-and-regularization/

### Decision trees for regression
The Decision tree method builds regression or classification models in the form of a tree structure. 
It brakes down a dataset into smaller and smaller subsets while incrementally developing an associated decision tree at the same time.
The final result is a tree with decision nodes and leaf nodes.

* A decision node has two or more branches each representing values for the attribute being tested.
* A leaf node represents a decision on the numerical target. 
* The topmost node is called the "root node".

Decision trees can handle both categorical and numerical data.
A decision tree is built top-down from a root node and involves partitioning the data into subsets that contain instances with similar values (homogenous).
We use standard deviation to calculate the homogeneity of a numerical sample.
If the numerical sample is completely homogeneous its standard deviation is zero.
The standard deviation reduction is based on the decrease in standard deviation after a dataset is split on an attribute. 
Constructing a decision tree is all about finding attribute that returns the highest standard deviation reduction (i.e., the most homogeneous branches).
I recommend reading this content for a more detailed explanation of this concept: 
http://chem-eng.utoronto.ca/~datamining/dmc/decision_tree_reg.htm

Just like using linear models for regression tasks involves changing the loss function used, using decision trees for regression involves changing the measure of the node impurity used.
The impurity metric is called variance and is defined in the same way as the squared loss for least squares linear regression.

## Extracting the right features from you data
As the underlying models for regression are the same as those for the classification case, we can use the same approach to create input features. 
The only practical difference is that the target is now a real-valued variable, as opposed to a categorical one. 
The `LabeledPoint` class in MLlib already takes this into account, as the label field is of the `Double` type, so it can handle both cases.

### Extracting features from the bike sharing dataset
To illustrate the concepts in this chapter, we will be using the bike sharing dataset. 
This dataset contains hourly records of the number of bicycle rentals in the capital bike sharing system. 
It also contains variables related to date and time, weather, and seasonal and holiday information.

Here are the variable names and descriptions:

* `instant`: This is the record ID
* `dteday`: This is the raw data
* `season`: This is different seasons such as spring, summer, winter, and fall
* `yr`: This is the year (2011 or 2012)
* `mnth`: This is the month of the year
* `hr`: This is the hour of the day
* `holiday`: This is whether the day was a holiday or not
* `weekday`: This is the day of the week
* `workingday`: This is whether the day was a working day or not
* `weathersit`: This is a categorical variable that describes the weather at a particular time
* `temp`: This is the normalized temperature
* `atemp`: This is the normalized apparent temperature
* `hum`: This is the normalized humidity
* `windspeed`: This is the normalized wind speed
* `cnt`: This is the target variable, that is, the count of bike rentals for that hour


In [2]:
path = "bike/hour_noheader.csv"
raw_data = sc.textFile(path)
num_data = raw_data.count()
records = raw_data.map(lambda x: x.split(","))
first = records.first()
print first
print num_data

[u'1', u'2011-01-01', u'1', u'0', u'1', u'0', u'0', u'6', u'0', u'1', u'0.24', u'0.2879', u'0.81', u'0', u'3', u'13', u'16']
17379

This code above loads the "bike/hour_noheader.csv" dataset into the Spark instance that is created (similar to previous datasets that we have used in earlier modules).
The data is then counted in the line `num_data = raw_data.count()`. 
The data is then parsed in this line `records = raw_data.map(lambda x: x.split(","))`. 

* The lambda expression splits each line in the data set by the comma (,) character and returns the list to be stored in the `records` variable.

The first line in the records dataset is selected in this line `first = records.first()`.
It is then printed and along with the number of records that was previously counted. 
As we can see from the results, we have 17,379 hourly records in our dataset. 
For now, we will ignore the record ID and raw date columns.
We will also ignore the `casual` and `registered` count target variables and focus on the overall count variable, `cnt` (which is the sum of the other two counts).
We are left with 12 variables. 
The first eight are categorical, while the last 4 are normalized real-valued variables.

To deal with the eight categorical variables, we will use the binary encoding approach.
The four real-valued variables will be left as is.

First, we will cache the dataset since we will be reading from it a lot in this lab.

In [3]:
records.cache()

PythonRDD[4] at RDD at PythonRDD.scala:43

In [ ]:
In order to extract each categorical feature into a binary vector form, we will need to know the feature mapping of each feature value to the index of the nonzero value in our binary vector.
So this means....what?

* We need to use a binary vector to store the codes for the eight categorical variabes the we get from our binary encoding method. (We haven't done this yet.)
* To perform this method, we have to know how each feature maps to an index within the binary vector. 

Let's define a function that will extract this mapping from our dataset for a given column.
The function below will do this.

In [6]:
def get_mapping(rdd, idx):
    return rdd.map(lambda fields: fields[idx]).distinct().zipWithIndex().collectAsMap()

The function `get_mapping(rdd, idx)` first maps the field to its unique values. 
Then, it uses the `zipWithIndex()` transformation to "zip" the value up with a unique index such that a key-value RDD is formed.
In the formulated RDD, the key is the variable and the value is the index.

* This index will be the index of the nonzero entry in the binary vector representation of the feature.

We then collect this RDD back to the driver as a Python dictionary via the `collectAsMap()` function.

Let's test our function.

In [7]:
print "Mapping of first categorical feature column: %s" % get_mapping(records, 2)

Mapping of first categorical feature column: {u'1': 0, u'3': 1, u'2': 2, u'4': 3}

Now, let's apply the function to each categorical column (i.e. for variable indices 2 to 9).

In [8]:
mappings = [get_mapping(records, i) for i in range(2,10)]
cat_len = sum(map(len, mappings))
num_len = len(records.first()[11:15])
total_len = num_len + cat_len

We now have the mappings for each variable (as seen in the code above), and we can see how many values in total we need for our binary vector representation in the code below.

In the code block above:

* `mappings = [get_mapping(records, i) for i in range(2,10)]` obtains the mappings that we desire for each variable.
* `cat_len = sum(map(len, mappings))` obtains the length of each variable's mapping, and then sums them.
* `num_len = len(records.first()[11:15])` obtains the length of the values in the 11th-15th indices in the first list in records
    * `first()` gets the first item from the `records` list
* `total_len = num_len + cat_len` calculates the total length of the values, which will be necessary for our tree construction

In [9]:
print "Feature vector length for categorical features: %d" % cat_len
print "Feature vector length for numerical features: %d" % num_len
print "Total feature vector length: %d" % total_len

Feature vector length for categorical features: 57
Feature vector length for numerical features: 4
Total feature vector length: 61